In [3]:
# Since all the continues features are bounded in certain range, we use the MinMaxScaler for the features to
# be proprocessed. Note that the values of feature "X1" and "X7" are already in the range of [0,1]
# no proprocessing is needed for these two featurs. 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

df = pd.read_csv("../raw/ENB2012_data.csv")
minmax_scaler = MinMaxScaler()
columns_minmax = ['X1','X2','X3','X4','X5','X6','X7', 'X8','Y1','Y2']
minmax_scaler.fit(df[columns_minmax])
df[columns_minmax] = minmax_scaler.transform(df[columns_minmax])
df.head()
df.to_csv("df_preprocessed_regression.csv")

In [24]:
df = pd.read_csv("../raw/ENB2012_data.csv")
y1_mean = df.Y1.mean()
y2_mean = df.Y2.mean()
# print(y1_mean, y2_mean)
# low loading -> high energy efficiency, high loading -> high energy efficiency
df['Y1'] = df['Y1'].apply(lambda x: 'high' if x <= y1_mean else 'low')
df['Y2'] = df['Y2'].apply(lambda x: 'high' if x <= y2_mean else 'low')
# df.loc[df.Y1 <= y1_mean, 'Y1'] = 'high'  # low loading means high energy efficiency
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,high,high
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,high,high
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,high,high
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,high,high
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,high,low


In [26]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
cat_ftrs = ['X6','X7', 'X8']
num_ftrs = ['X1','X2','X3','X4','X5',]
label_ftrs = ['Y1', 'Y2']

# one-hot encoder 
ohe = OneHotEncoder(sparse=False)
cat_values = ohe.fit_transform(df[cat_ftrs])
cat_ftr_names = ohe.get_feature_names()
df_cat = pd.DataFrame(data=cat_values,columns = cat_ftr_names)

# minmax scaler
minmax = MinMaxScaler()
num_values = minmax.fit_transform(df[num_ftrs])
df_num = pd.DataFrame(data=num_values,columns = num_ftrs) 

# label encoder
le = LabelEncoder()
label_value_heat = le.fit_transform(df['Y1'])
label_value_cool = le.fit_transform(df['Y2'])
df_label_heat = pd.DataFrame(data=label_value_heat, columns=['Y1'])
df_label_cool = pd.DataFrame(data=label_value_cool, columns=['Y2'])

# concatenation of all features
df_preprocessed = pd.concat([df_cat, df_num, df_label_heat, df_label_cool], axis=1)
df_preprocessed.to_csv("df_preprocessed_classification.csv")
df_preprocessed.shape

/usr/lib64/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(768, 18)